In [1]:
from lang import load_data_int_seq
from models.rl.env import IntegerSequenceEnv
from models.rl.agents.agent_mcts.main import train_mcts


%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "penalty_at_end": True})

In [ ]:
train_mcts(env, num_epochs=200, policy=None)

TypeError: super(type, obj): obj must be an instance or subtype of type

# HillClimbingEnv

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

print(output)
print(torch.sum(output, dim=1))

tensor([[0.0529, 0.6313, 0.3158],
        [0.2920, 0.6516, 0.0564]])
tensor([1., 1.])


In [6]:
from models.rl.agents.agent_mcts.env_archive.hill_climbing_env import HillClimbingEnv

In [7]:
env = HillClimbingEnv()

In [8]:
train_mcts(env, num_epochs=2)

NameError: name 'train_mcts' is not defined

In [36]:
import matplotlib.pyplot as plt
value_losses = [8.120367050170898, 0.0, 1.0, 9.726750373840332, 7.819188117980957, 6.215205669403076, 4.929923057556152, 3.938908815383911, 3.1917002201080322, 2.631671667098999, 2.209272623062134, 1.8863438367843628, 1.6352969408035278, 1.4366923570632935, 1.276902198791504, 1.1463127136230469, 1.0380536317825317, 0.9471443295478821, 0.869920551776886, 0.8036395907402039, 0.7462201118469238, 0.6960629224777222, 0.6519147753715515, 0.6127952933311462, 0.5779146552085876, 0.5466380715370178, 0.5184493660926819, 0.4929257035255432, 0.4697134792804718, 0.4485212564468384, 0.42909908294677734, 0.41124123334884644, 0.39476874470710754, 0.37952926754951477, 0.36539238691329956, 0.35224491357803345, 0.33998775482177734, 0.3285349905490875, 0.31781110167503357, 0.30774909257888794, 0.2982923686504364, 0.289387047290802, 0.2809865176677704, 0.27305158972740173, 0.26554322242736816, 0.25843000411987305, 0.25168073177337646, 0.24526852369308472, 0.23916971683502197, 0.23336268961429596, 0.22782453894615173, 0.2225402295589447, 0.21749117970466614, 0.21266265213489532, 0.20803986489772797, 0.20361250638961792, 0.1993662416934967, 0.195291206240654, 0.19137634336948395, 0.18761339783668518, 0.1839941442012787, 0.18051081895828247, 0.17715393006801605, 0.17391857504844666, 0.1707974672317505, 0.16778506338596344, 0.1648758202791214, 0.16206586360931396, 0.15934786200523376, 0.15671849250793457, 0.15417328476905823, 0.15170888602733612, 0.14932137727737427, 0.14700622856616974, 32.23725128173828, 1.0, 30.925750732421875, 0.24039007723331451, 0.23451849818229675, 0.22892242670059204, 0.2235817015171051, 27.73172378540039, 23.82838249206543, 0.6066253185272217, 0.5720002055168152, 0.5409716367721558, 0.5130198001861572, 0.48772090673446655, 0.46472227573394775, 0.443731427192688, 0.42449864745140076, 0.406821072101593, 0.390516459941864, 0.37543758749961853, 0.3614499568939209, 0.3484443426132202, 0.33632004261016846, 0.32499343156814575, 0.31438958644866943, 0.3044406771659851]
policy_losses = [8.120367050170898, 0.0, 2.0, 9.726750373840332, 7.819188117980957, 6.215205669403076, 4.929923057556152, 3.938908815383911, 3.1917002201080322, 2.631671667098999, 2.209272623062134, 1.8863438367843628, 1.6352969408035278, 1.4366923570632935, 1.276902198791504, 1.1463127136230469, 1.0380536317825317, 0.9471443295478821, 0.869920551776886, 0.8036395907402039, 0.7462201118469238, 0.6960629224777222, 0.6519147753715515, 0.6127952933311462, 0.5779146552085876, 0.5466380715370178, 0.5184493660926819, 0.4929257035255432, 0.4697134792804718, 0.4485212564468384, 0.42909908294677734, 0.41124123334884644, 0.39476874470710754, 0.37952926754951477, 0.36539238691329956, 0.35224491357803345, 0.33998775482177734, 0.3285349905490875, 0.31781110167503357, 0.30774909257888794, 0.2982923686504364, 0.289387047290802, 0.2809865176677704, 0.27305158972740173, 0.26554322242736816, 0.25843000411987305, 0.25168073177337646, 0.24526852369308472, 0.23916971683502197, 0.23336268961429596, 0.22782453894615173, 0.2225402295589447, 0.21749117970466614, 0.21266265213489532, 0.20803986489772797, 0.20361250638961792, 0.1993662416934967, 0.195291206240654, 0.19137634336948395, 0.18761339783668518, 0.1839941442012787, 0.18051081895828247, 0.17715393006801605, 0.17391857504844666, 0.1707974672317505, 0.16778506338596344, 0.1648758202791214, 0.16206586360931396, 0.15934786200523376, 0.15671849250793457, 0.15417328476905823, 0.15170888602733612, 0.14932137727737427, 0.14700622856616974, 32.23725128173828, 1.0, 30.925750732421875, 0.24039007723331451, 0.23451849818229675, 0.22892242670059204, 0.2235817015171051, 27.73172378540039, 23.82838249206543, 0.6066253185272217, 0.5720002055168152, 0.5409716367721558, 0.5130198001861572, 0.48772090673446655, 0.46472227573394775, 0.443731427192688, 0.42449864745140076, 0.406821072101593, 0.390516459941864, 0.37543758749961853, 0.3614499568939209, 0.3484443426132202, 0.33632004261016846, 0.32499343156814575, 0.31438958644866943, 0.3044406771659851]

# plt.plot(policy_losses)

fig, axs = plt.subplots(3, 1)
            

axs[0].plot(policy_losses)

# axs[0].plot(value_losses)
# axs[0].set_xlabel("epochs")
# axs[0].set_ylabel("value loss")

# axs[1].plot(policy_losses)
# axs[1].set_xlabel("epochs")
# axs[1].set_ylabel("policy loss")


# axs[2].plot(total_rews)
# axs[2].set_xlabel("epochs")
# axs[2].set_ylabel("testing loss")

fig.savefig("training_{}.jpg".format("testing numbers2"))

In [28]:
fig, axs = plt.subplots(1, 2, constrained_layout=True)

In [29]:
ax = axs[0]
ax.plot(range(10))

In [31]:
ax.show()

AttributeError: 'AxesSubplot' object has no attribute 'show'